# DATA 620 - Assignment 6

Jeremy OBrien, Mael Illien, Vanita Thompson

## Document Classification

* It can be useful to be able to classify new "test" documents using already classified "training" documents. A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam. Here is one example of such data:  UCI Machine Learning Repository: Spambase Data Set (http://archive.ics.uci.edu/ml/datasets/Spambase)
* For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).
* For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.


Resources:

- http://www.cs.ucf.edu/courses/cap5636/fall2011/nltk.pdf
- https://bbengfort.github.io/tutorials/2016/05/19/text-classification-nltk-sckit-learn.html
- https://www.cs.bgu.ac.il/~elhadad/nlp16/spam_classifier.html

## Setup

From spambase documentation:

Number of Instances: 4601 (1813 Spam = 39.4%)
    
Number of Attributes: 58 (57 continuous, 1 nominal class label)

In [1]:
import csv
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## Data Import

In [2]:
# Read csv data
data = []

with open('./spambase/spambase.data') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [3]:
len(data)

4601

In [4]:
# The last items of each row list contains the label of the row
X = np.array([x[:-1] for x in data]).astype(np.float)
y = np.array([x[-1] for x in data]).astype(np.float)

## Data Transformation

The simple approach taken is case normalization, stopword remova, and stemming, then TF-IDF vectorization (apparently tokenizing doesn’t work well with email due to colloquial speech)

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

### Naive Bayes - Gaussian

In [ ]:
# Instantiate and train Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X_train, y_train)
# Predict on test set
y_pred = gnb.predict(X_test)

In [ ]:
gnb.score(X_train,y_train)

In [ ]:
gnb.score(X_test, y_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Naive Bayes - Bernoulli

In [ ]:
# Instantiate and train Gaussian Naive Bayes model
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
# Predict on test set
y_pred = bnb.predict(X_test)

In [ ]:
bnb.score(X_train,y_train)

In [ ]:
bnb.score(X_test, y_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Conclusion

## Youtube